In [3]:
import pandas as pd

with open("./avg-test-result-in-paper", "r") as f:
    df = pd.read_csv(f, delim_whitespace=True, engine='python')
df.head()

,file_system,dup_rate,num_job,bandwidth(MiB/s)
0,Light-Dedup,0,1,778
1,Light-Dedup,0,2,1233
2,Light-Dedup,0,4,1667
3,Light-Dedup,0,8,1793
4,Light-Dedup,0,16,1563


- How much Light-Dedup outperform other deduplication file systems under 0, 25, 50, 75 duplication ratio?

In [4]:
fs_names = df['file_system'].drop_duplicates().reset_index().drop(columns=['index'])['file_system']

light_dedup =  df[df["file_system"] == "Light-Dedup"].copy().reset_index()

for fs_name in fs_names:
    if fs_name == "Light-Dedup":
        continue
    fs = df[df["file_system"] == fs_name]
    # print(pd.DataFrame([1, 2, 3]) / pd.DataFrame([1, 2, 3]))
    light_dedup[fs_name] = light_dedup["bandwidth(MiB/s)"] / fs["bandwidth(MiB/s)"].reset_index()["bandwidth(MiB/s)"]

light_dedup
    

,index,file_system,dup_rate,num_job,bandwidth(MiB/s),Light-Dedup(SHA256),NV-Dedup,NOVA
0,0,Light-Dedup,0,1,778,2.768683,0.972500,0.900463
1,1,Light-Dedup,0,2,1233,2.357553,0.950655,0.911982
2,2,Light-Dedup,0,4,1667,1.809989,1.011529,0.956946
3,3,Light-Dedup,0,8,1793,1.567308,1.178055,0.960364
4,4,Light-Dedup,0,16,1563,1.158636,1.178733,1.005145
5,20,Light-Dedup,25,1,690,2.429577,1.292135,0.817536
6,21,Light-Dedup,25,2,1206,2.258427,1.273495,0.900000
7,22,Light-Dedup,25,4,1887,1.904137,1.288934,1.080137
8,23,Light-Dedup,25,8,2091,1.391218,1.271898,1.121180
9,24,Light-Dedup,25,16,1670,0.969240,1.158917,1.075338


- How much throughput Light-Dedup achieves over NOVA and NV-Dedup when dedup ratio >= 25%?

In [5]:
print("Max higher than NV-Dedup:" + str(max(light_dedup[light_dedup["dup_rate"] >= 25]["NV-Dedup"])))
print("Min higher than NV-Dedup:" + str(min(light_dedup[light_dedup["dup_rate"] >= 25]["NV-Dedup"])))
print("Max higher than NOVA:" + str(max(light_dedup[light_dedup["dup_rate"] >= 25]["NOVA"])))
print("Min higher than NOVA:" + str(min(light_dedup[light_dedup["dup_rate"] >= 25]["NOVA"])))

Max higher than NV-Dedup:1.9931972789115646
Min higher than NV-Dedup:1.130926724137931
Max higher than NOVA:2.173076923076923
Min higher than NOVA:0.7902097902097902


- How much slower Light-Dedup than NOVA when dedup ratio == 0%?

In [6]:
light_dedup[light_dedup["dup_rate"] == 0]["NOVA"]

0    0.900463
1    0.911982
2    0.956946
3    0.960364
4    1.005145
Name: NOVA, dtype: float64

- How much faster Light-Dedup than NV-Dedup when dedup ratio == 75%?

In [7]:
light_dedup[light_dedup["dup_rate"] == 75]["NV-Dedup"]

15    1.901961
16    1.993197
17    1.957664
18    1.828315
19    1.324980
Name: NV-Dedup, dtype: float64

- How much faster Light-Dedup than SHA256 when dedup ratio == 75%?

In [8]:
light_dedup[(light_dedup["dup_rate"] == 75) & (light_dedup["num_job"] == 1)]["Light-Dedup(SHA256)"]

15    2.080429
Name: Light-Dedup(SHA256), dtype: float64

- Performance Improvement of Light-Dedup from 0% to 75% 

In [11]:
light_dedup[(light_dedup["dup_rate"] == 75)].reset_index()["bandwidth(MiB/s)"]/light_dedup[(light_dedup["dup_rate"] == 0)].reset_index()["bandwidth(MiB/s)"]

0    0.997429
1    1.188159
2    1.608878
3    2.268823
4    2.165067
Name: bandwidth(MiB/s), dtype: float64